In [4]:
%matplotlib inline

import gym
from gym.wrappers import Monitor
import itertools
import numpy as np
import os
import random
import sys
import psutil
import tensorflow as tf

if "../" not in sys.path:
    sys.path.append("../")

from libr import plotting    
from collections import deque, namedtuple
from matplotlib import pyplot as plt

In [5]:
env = gym.envs.make("Breakout-v0")

In [ ]:
# 4 valid actions are available
valid_actions = [0,1,2,3]

In [ ]:
class StateProcessor():
    """
    Resize and grascale image. 
    """
    def __init__(self):
        # Build the tensorflow graph
        with tf.variable_scope("state_processor"):
            